Making Wiki tool

In [65]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia


WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/opt/anaconda3/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

Loading text and saving in text documents  from the link 

In [66]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader=WebBaseLoader(web_paths=("https://www.ibm.com/topics/artificial-intelligence",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                      class_=("rich-text text")    
                     )))
text_documents2=loader.load()
text_documents2[0].page_content.splitlines()[:5]

['',
 'Artificial intelligence (AI) is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision making, creativity and autonomy.',
 '',
 'Applications and devices equipped with AI can see and identify objects. They can understand and respond to human language. They can learn from new information and experience. They can make detailed recommendations to users and experts.\xa0They can act independently, replacing the need for human intelligence or intervention (a classic example being a self-driving car).',
 'But in 2024, most AI researchers, practitioners and most AI-related headlines are focused on breakthroughs in generative AI\xa0(gen AI), a technology that can create original text, images, video and other content. To fully understand generative AI, it’s important to first understand the technologies on which generative AI tools are built: machine learning\xa0(ML) and deep learning.']

Splitting the text documents in small parts

In [67]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(text_documents2)
documents[:1]

[Document(metadata={'source': 'https://www.ibm.com/topics/artificial-intelligence'}, page_content='Artificial intelligence (AI) is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision making, creativity and autonomy.\n\nApplications and devices equipped with AI can see and identify objects. They can understand and respond to human language. They can learn from new information and experience. They can make detailed recommendations to users and experts.\xa0They can act independently, replacing the need for human intelligence or intervention (a classic example being a self-driving car).\nBut in 2024, most AI researchers, practitioners and most AI-related headlines are focused on breakthroughs in generative AI\xa0(gen AI), a technology that can create original text, images, video and other content. To fully understand generative AI, it’s important to first understand the technologies on which generative AI tools are built: mach

Embedding the small ext documents and saving them in vector store 

In [68]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
database=FAISS.from_documents(documents,HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
retriever=database.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x3cb8691f0>, search_kwargs={})

Creating Artificial Intelligence search tool using retriever

In [69]:
from langchain.tools.retriever import create_retriever_tool
AI_search=create_retriever_tool(retriever,"Artificial Intelligence Search","If I ask any question realted to artificial Intelligence use this tool")
AI_search.name

'Artificial Intelligence Search'

Making Arxiv Tool 

In [70]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=150)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

Combining tools

In [71]:
tools=[wikipedia,AI_search,arxiv]

Using llama2 llm with the help of ollama

In [72]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama2")
llm

Ollama()

Agents

In [73]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

Creating prompt

In [74]:

from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(""" Answer to the my querys using context and also give  the source of that in your answer
<context>
{context}
</context>
Question:{input}
{agent_scratchpad}                                 
                                           """

 )

In [75]:
from langchain.agents import AgentExecutor
agent_excutor=AgentExecutor(agent=agent,tools=tools,verboe=True)
agent_excutor

AgentExecutor(verbose=False, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'context', 'input'], input_types={}, partial_variables={}, template=' Answer to the my querys using context and also give  the source of that in your answer\n<context>\n{context}\n</context>\nQuestion:{input}\n{agent_scratchpad}                                 \n                                           '), additional_kwargs={})])
| RunnableBinding(bound=Ollama(), kwargs={'tools': [{'type': 'function', 'function': {'name': 'wikipedia', 'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical 

In [76]:
agent_excutor.invoke({"input":"What is artificial intelligence","context": "" })

OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama2`.